# SQLAlchemy

The SQLAlchemy models allows for the translation between OO languages like python and relational languages like SQL. It transforms python objects into database information, and the other way around. PSQLAlcheny can connect to many differnt types of sql databases, but all of them follow this standard. The standard specifies the functions that help you connect to a database, send queries, and get results. All the specifications are regulated by PEP 249.

In [3]:
!pip install sqlalchemy

For this topic we will practic on the `Buildings_Database.sqlite`. using the folowing code we can connect to the databse.

In [4]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Buildings_Database.sqlite', echo=True)
connection = engine.connect()
print(connection, engine)

<sqlalchemy.engine.base.Connection object at 0x000001BB6B8077F0> Engine(sqlite:///Buildings_Database.sqlite)


In [5]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

2023-04-27 21:05:48,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:48,794 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-04-27 21:05:48,795 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:48,798 INFO sqlalchemy.engine.Engine ROLLBACK


[]

!! other code in topic https://hyperskill.org/learn/step/12935 that i did not get working

## Data mapping

Mapping transforms SQL objects into Python ones. There are two main mapping types in SQLAlchemy. One is **classical** mapping, the other is **declarative** mapping. We will discuss these types below.

### Classical

Classical mapping refers to the configuration of **a mapped class** that was created with the `mapper()` function. We need to define a database table and the corresponding Python class separately to link them with `mapper()`. After that, all changes to the table and class made via SQLAlchemy are saved in your database. Classical mapping is a **base mapping system** provided by the ORM. Take a look at the snippet below:

````python
from sqlalchemy import Table, MetaData, Column, Integer, String
from sqlalchemy.orm import mapper

metadata = MetaData()

animals = Table('animals', metadata,
                Column('id', Integer, primary_key=True),
                Column('petname', String(30)),
                Column('age', Integer),
                Column('weight', Integer))


class Animals(object):
    def __init__(self, petname, age, weight):
        self.petname = petname
        self.age = age
        self.weight = weight


mapper(Animals, animals)
````

It however does not seem to work with SQLAlchemy 2.0. Must look up right topic

### Declarative

Declarative mapping is a concise version of classical mapping. We don't need to specify a class and a table separately, we can do it all in one class. Let's try to write a concise form of the previous snippet:

!! Had to be changed to match with **SQLAlchemy 2.0**: `from sqlalchemy.orm import declarative_base`

In [6]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()


class Animals(Base):
    __tablename__ = 'animals'

    id = Column(Integer, primary_key=True)
    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

In [7]:
Base.metadata.create_all(engine)

2023-04-27 21:05:48,836 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:48,837 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-04-27 21:05:48,838 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:48,839 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-04-27 21:05:48,840 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:48,842 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-04-27 21:05:48,843 INFO sqlalchemy.engine.Engine [no key 0.00104s] ()
2023-04-27 21:05:48,854 INFO sqlalchemy.engine.Engine COMMIT


### Session
The `mapper()` function and declarative extensions are primary interfaces for ORM. Once our mappings are configured, we can proceed to the primary interface. It is also known as a session. It helps us to communicate with our database and ensures that all operations run smoothly. You can modify your database and save the changes during the session. To start one, you can use the following statement:

In [8]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session

In [9]:
animal_1 = Animals(petname='Timmy', age=2, weight=12)
animal_2 = Animals(petname='Tommy', age=3, weight=10)
animal_3 = Animals(petname='Kitty', age=4, weight=14)
session.add(animal_1)
session.add(animal_2)
session.add(animal_3)
session.commit()

2023-04-27 21:05:48,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:48,891 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-04-27 21:05:48,893 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues)] ('Timmy', 2, 12, 'Tommy', 3, 10, 'Kitty', 4, 14)
2023-04-27 21:05:48,897 INFO sqlalchemy.engine.Engine COMMIT


## querying and filtering

 A Query object will generate a SELECT statement for columns of our table and rename each column to a variable name as per PEP convention. Here's what it may look like. Once passed, it will create the animals_petname variable for the table called Animals with the petname column. After this, you'll be able to access the values in the corresponding column via this variable.

In [10]:
from sqlalchemy.orm import Query
query = Query(Animals)
query

In [11]:
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [12]:
from sqlalchemy.orm import Query, sessionmaker
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-04-27 21:05:48,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:48,949 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-04-27 21:05:48,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:48,952 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-04-27 21:05:48,952 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:48,953 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-04-27 21:05:48,955 INFO sqlalchemy.engine.Engine [no key 0.00111s] ()
2023-04-27 21:05:48,956 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
query = session.query(Animals)
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [14]:
session.add(Animals(petname="Billy", age=1, weight=8))
session.add(Animals(petname="Susan", age=4, weight=12))

session.commit()

2023-04-27 21:05:48,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:48,993 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?), (?, ?, ?) RETURNING id
2023-04-27 21:05:48,994 INFO sqlalchemy.engine.Engine [generated in 0.00013s (insertmanyvalues)] ('Billy', 1, 8, 'Susan', 4, 12)
2023-04-27 21:05:48,996 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
all_rows = query.all()

2023-04-27 21:05:49,005 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:49,009 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals
2023-04-27 21:05:49,010 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ()


In [16]:
for row in all_rows:
    print(type(row))  # <class '__main__.Animals'>

<class '__main__.Animals'>
<class '__main__.Animals'>


In [17]:
for row in all_rows:
    print(f"Pet name: {row.petname}, age: {row.age}, weight: {row.weight}")

Pet name: Billy, age: 1, weight: 8
Pet name: Susan, age: 4, weight: 12


In [18]:
query = session.query(Animals.petname)
print(query)

SELECT animals.petname AS animals_petname 
FROM animals


In [19]:
query = session.query(Animals.petname, Animals.age)
for petname, age in query:
    print(petname, age)

2023-04-27 21:05:49,068 INFO sqlalchemy.engine.Engine SELECT animals.petname AS animals_petname, animals.age AS animals_age 
FROM animals
2023-04-27 21:05:49,069 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ()
Billy 1
Susan 4


In [20]:
query = session.query(Animals)
query.count()

2023-04-27 21:05:49,085 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals) AS anon_1
2023-04-27 21:05:49,086 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ()


2

In [21]:
query = session.query(Animals)
for row in query.filter(Animals.petname == "Billy"):
    print(row.petname, row.age, row.weight) # Billy 1 8

2023-04-27 21:05:49,102 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.petname = ?
2023-04-27 21:05:49,103 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('Billy',)
Billy 1 8


In [22]:
query = session.query(Animals.age, Animals.weight)

age_gr_than = Animals.age > 2
weight_eq_gr_than = Animals.weight >= 8

for age, weight in query.filter(age_gr_than, weight_eq_gr_than):
    print(f"Pet age: {age}, Pet weight: {weight}")

2023-04-27 21:05:49,119 INFO sqlalchemy.engine.Engine SELECT animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.age > ? AND animals.weight >= ?
2023-04-27 21:05:49,120 INFO sqlalchemy.engine.Engine [generated in 0.00124s] (2, 8)
Pet age: 4, Pet weight: 12


## Updating and deleting

### imports and table creation

In [23]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import DeclarativeBase, sessionmaker

Base = declarative_base()

In [24]:
class Employee(Base):
    __tablename__ = "employee"

    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    position = Column(String(30))
    salary = Column(Integer)

### Create DB and session

In [25]:
engine = create_engine("sqlite:///alchemy.db", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-04-27 21:05:49,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:05:49,162 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2023-04-27 21:05:49,163 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:49,165 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2023-04-27 21:05:49,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-27 21:05:49,166 INFO sqlalchemy.engine.Engine 
CREATE TABLE employee (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	position VARCHAR(30), 
	salary INTEGER, 
	PRIMARY KEY (id)
)


2023-04-27 21:05:49,167 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2023-04-27 21:05:49,179 INFO sqlalchemy.engine.Engine COMMIT


### Fill the session

In [27]:
session.add(Employee(name="William", position="Data Scientist", salary="12,000"))
session.add(Employee(name="Susan", position="Product Manager", salary="14,000"))
session.add(Employee(name="Andrew", position="Senior Software Engineer", salary="17,000"))
session.add(Employee(name="Nancy", position="Account Executive", salary="13,000"))

session.commit()

2023-04-27 21:07:15,114 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:07:15,127 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, position, salary) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-04-27 21:07:15,127 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues)] ('William', 'Data Scientist', '12,000', 'Susan', 'Product Manager', '14,000', 'Andrew', 'Senior Software Engineer', '17,000', 'Nancy', 'Account Executive', '13,000')
2023-04-27 21:07:15,127 INFO sqlalchemy.engine.Engine COMMIT


### Update methode

In [28]:
query = session.query(Employee)

query.update({
    "salary": Employee.salary + 1000
})

session.commit()

2023-04-27 21:16:22,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:16:22,700 INFO sqlalchemy.engine.Engine UPDATE employee SET salary=(employee.salary + ?)
2023-04-27 21:16:22,701 INFO sqlalchemy.engine.Engine [generated in 0.00119s] (1000,)
2023-04-27 21:16:22,704 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
empl_filter = query.filter(Employee.name == "William")
empl_filter.update({"salary": Employee.salary + 1000})
session.commit()

2023-04-27 21:20:42,456 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:20:42,456 INFO sqlalchemy.engine.Engine UPDATE employee SET salary=(employee.salary + ?) WHERE employee.name = ?
2023-04-27 21:20:42,456 INFO sqlalchemy.engine.Engine [generated in 0.00081s] (1000, 'William')
2023-04-27 21:20:42,456 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
empl_filter = query.filter(Employee.position == "Senior Software Engineer")
empl_filter.update({
    "salary": Employee.salary - 1500,
    "position": "Middle Software Engineer"
})
session.commit()

2023-04-27 21:22:09,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:22:09,979 INFO sqlalchemy.engine.Engine UPDATE employee SET position=?, salary=(employee.salary - ?) WHERE employee.position = ?
2023-04-27 21:22:09,979 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ('Middle Software Engineer', 1500, 'Senior Software Engineer')
2023-04-27 21:22:09,979 INFO sqlalchemy.engine.Engine COMMIT


### Deleting data

In [32]:
query.filter(Employee.name == "Nancy").delete()
session.commit()

2023-04-27 21:23:18,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-27 21:23:18,123 INFO sqlalchemy.engine.Engine DELETE FROM employee WHERE employee.name = ?
2023-04-27 21:23:18,123 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ('Nancy',)
2023-04-27 21:23:18,123 INFO sqlalchemy.engine.Engine COMMIT
